# Mu Numerical Residuals

This notebook demonstrates residuals between $\mu$ values calculated by `astopy` and `sncosmo`.

In [ ]:
from pathlib import Path

import numpy as np
import sncosmo
from astropy.cosmology import WMAP9
from matplotlib import pyplot as plt

fig_dir = Path('.') / 'figs'
fig_dir.mkdir(exist_ok=True, parents=True)


In [ ]:
model = sncosmo.Model('salt2-extended')
b_band = sncosmo.get_bandpass('standard::b')
z_vals = np.arange(.01, 1.1, .01)
peak_mag = -19

apparent_mag = []
for z in z_vals:
    model.set(z=z)
    model.set_source_peakabsmag(peak_mag, b_band, 'AB')

    rest_band = b_band.shifted(1 + z)
    mag = model.bandmag(rest_band, 'ab', 0) 
    
    apparent_mag.append(mag)

mu = apparent_mag + 2.5 * np.log10(1 + z_vals) - peak_mag


In [ ]:
cosmo_mu = WMAP9.distmod(z_vals)
plt.plot(z_vals, mu - cosmo_mu.value, label='WMAP9')

plt.ylabel(r'$\mu$ - WMAP9')
plt.xlabel('z')
plt.legend()

plt.savefig(fig_dir / 'mu_residuals.pdf')
